<a href="https://colab.research.google.com/github/anandaptralmira/textmining/blob/master/Text_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install Sastrawi

In [ ]:
# Import Library
import pandas as pd
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import string 
import re
import nltk
from google.colab import files

nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

from collections import Counter
from wordcloud import STOPWORDS
from wordcloud import WordCloud

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import re
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/anandaptralmira/textmining/master/id-hatespeech-detection-master.csv' , sep=",", encoding="ISO-8859-1")
df.head()

In [ ]:
df.info()

In [ ]:
# Set Training and Testing Data (75:25)
X_train, X_test, y_train, y_test  = train_test_split(df['Tweet'] , df['Label'], shuffle = True, test_size=0.25, random_state=1)

# Show the Training and Testing Data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#cek proporsi label data
pd.value_counts(df['Label']).plot.bar()

In [13]:
df_fix = df.append(df, ignore_index=True)

In [ ]:
df_fix

In [ ]:
from wordcloud import WordCloud,STOPWORDS
word = df.Tweet[df['Label']=='HS'].astype('string')
wordcloud = WordCloud(max_font_size=30, max_words=500, background_color="black").generate_from_text(' '.join(word))
print('Hate speech')
plt.figure(figsize= (12,8))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
from wordcloud import WordCloud,STOPWORDS
word = df.Tweet[df['Label']=='Non_HS'].astype('string')
wordcloud = WordCloud(max_font_size=30, max_words=500, background_color="black").generate_from_text(' '.join(word))
print('Non Hate speech')
plt.figure(figsize= (12,8))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [20]:
# Cleaning the tweets
def preprocessing(a,b):
    data_fix = []
    from nltk.corpus import stopwords
    for i in range(len(b)):
        teks = re.sub('[^a-zA-Z]', ' ', a[i])
        # Remove mentions
        teks = re.sub(r'@[A-Za-z0-9_]+', '', a[i])
        # Remove hashtags
        teks = re.sub(r'#', '', a[i])
        # Remove retweets:
        teks = re.sub(r'RT : ', '', a[i])
        # Remove urls
        teks = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', a[i])
        #Case Folding
        teks = teks.lower()
        #Tokinize
        teks = teks.split()
        #Stopword + Stemming
        stemmer = StemmerFactory().create_stemmer()        
        teks = [stemmer.stem(word) for word in teks if not word in set(stopwords.words('indonesian'))]
        teks = [word for word in teks if len (word) > 3]
        teks = ' '.join(teks)
        data_fix.append(teks)
    return data_fix

In [23]:
panjang_set = df_fix['Label'].tolist()

In [24]:
data_set = preprocessing(df_fix['Tweet'],panjang_set)

In [25]:
df_fix['Prepro'] = pd.Series(data_set).values

In [ ]:
df_fix

In [30]:
# Import Module
from sklearn import svm

# Modeling Random Forest
esvm = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', probability=True)
esvm.fit(X_train, y_train)

# Predict to Test Data
y_pred_esvm= esvm.predict(X_test)

ValueError: ignored